In [ ]:
import numpy as np
import xgboost as xgb
import pandas as pd
import matplotlib.pyplot as plt

from matplotlib import style
from sklearn.metrics import accuracy_score
from sklearn import metrics
import seaborn as sns
sns.set_style('whitegrid')
import tensorflow as tf
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.model_selection import (GridSearchCV, RandomizedSearchCV,
                                     cross_val_score, train_test_split)
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
# import category_encoders as ce
import time
import warnings
import pickle

In [ ]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier

In [ ]:
df = pd.read_csv("dry_bulk_training_data.csv", skiprows=0)
df.head()

In [ ]:
df.arrival_port.value_counts()

In [ ]:
df.sub_segment.value_counts()

In [ ]:
df.departure_port.value_counts()

In [ ]:
df.sspd_mstd.value_counts()

In [ ]:
plt.figure(figsize=(40,20))
sns.countplot(x="arrival_port", data=df).set_title('Distribution of Arrival Ports')

In [ ]:
df.groupby(['departure_port','arrival_port']).size()

In [ ]:
df.groupby(['departure_port','arrival_port']).size().sort_values(ascending=False)[:12000].sum()/len(df)

In [ ]:
df.groupby(['departure_port','arrival_port']).size().sort_values(ascending=False)[:9000]

In [ ]:
df2 = df.groupby(['departure_port','arrival_port']).filter(lambda x: len(x) > 4)
df2.head()

In [ ]:
plt.figure(figsize=(40,20))
sns.countplot(x="arrival_port", data=df2).set_title('Distribution of Arrival Ports')

In [ ]:
df2.arrival_port.value_counts()

In [ ]:
df2.arrival_port.value_counts()[:700].values.sum()/len(df2)

In [ ]:
df2.arrival_port.value_counts()[:700]

In [ ]:
removals = df2['arrival_port'].value_counts().reset_index()
removals = removals[removals['arrival_port'] > 79]['index'].values
removals.size
df3 = df2[df2['arrival_port'].isin(removals)]
df3.arrival_port.value_counts()


In [ ]:
plt.figure(figsize=(40,20))
sns.countplot(x="arrival_port", data=df3).set_title('Distribution of Arrival Ports')

In [ ]:
df3.to_csv(r'dry_bulk_trained_data.csv', index = False)

In [ ]:
X=df3.drop('arrival_port',axis=1).copy()
X.head()

In [ ]:

X.drop(['voyage_id', 'imo','mstd_id'], axis=1, inplace=True)

In [ ]:
y = df3['arrival_port'].copy()
print(X.shape)
print(y.shape)

In [ ]:
le = LabelEncoder()
y_encoded= le.fit(y)
filehandler = open("enc_targets.obj","wb")
pickle.dump(y_encoded,filehandler)
filehandler.close()

In [ ]:
y_changed = y_encoded.transform(y)
y_changed

In [ ]:
dp = LabelEncoder()
dp_enc = dp.fit(X['departure_port'])
filehandler = open("dp_enc.obj","wb")
pickle.dump(dp_enc,filehandler)
filehandler.close()
#########
sea = LabelEncoder()
sea_enc = sea.fit(X['season'])
filehandler = open("sea_enc.obj","wb")
pickle.dump(sea_enc,filehandler)
filehandler.close()
###########
seg = LabelEncoder()
seg_enc = seg.fit(X['sub_segment'])
filehandler = open("seg_enc.obj","wb")
pickle.dump(seg_enc,filehandler)
filehandler.close()
###############
sspd = LabelEncoder()
sspd_enc = sspd.fit(X['sspd_mstd'])
filehandler = open("sspd_enc.obj","wb")
pickle.dump(sspd_enc,filehandler)
filehandler.close()

In [ ]:
file = open("dp_enc.obj",'rb')
dp_enc = pickle.load(file)
file.close()
X["departure_port"] = dp_enc.transform(X["departure_port"])
############
file = open("sea_enc.obj",'rb')
sea_enc = pickle.load(file)
file.close()
X["season"] = sea_enc.transform(X["season"])
##############
file = open("seg_enc.obj",'rb')
seg_enc = pickle.load(file)
file.close()
X["sub_segment"] = seg_enc.transform(X["sub_segment"])
#######################
file = open("sspd_enc.obj",'rb')
sspd_enc = pickle.load(file)
file.close()
X["sspd_mstd"] = sspd_enc.transform(X["sspd_mstd"])

In [ ]:
X.head()

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y_changed,test_size = 0.2, random_state=42)

In [ ]:
clf = xgb.XGBClassifier(objective='multi:softmax',seed=42,learn_rate=0.1,max_depth=6,gamma=0.29,subsample=0.9)#colsample_bytree=0.5
clf.fit(X_train,
        y_train, 
        verbose=True,
        eval_metric = ['merror','mlogloss'],
        early_stopping_rounds=12,
        eval_set=[(X_test,y_test),(X_train, y_train)])

In [ ]:
clf.best_ntree_limit

In [ ]:
import pickle
pickle.dump(clf, open("dty_bulk_model.dat", "wb"))

In [ ]:
def print_feature_importance(classifier,X):
        # check Important features
        feature_importances_df = pd.DataFrame(
            {"feature": list(X.columns),
             "importance": classifier.feature_importances_}
        ).sort_values("importance", ascending=False)

        # Display
        print("\n[XGBoostClassifier] feature importance:")
        print(feature_importances_df)

In [ ]:
features = print_feature_importance(clf, X)
print(features)

In [ ]:
y_pred = clf.predict(X_test) 

In [ ]:
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions) 

print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
import pickle
loaded_model = pickle.load(open("dry_bulk_model.dat", "rb"))
results = loaded_model.evals_result()
epochs = len(results['validation_0']['merror'])
x_axis = range(0, epochs)
from matplotlib import pyplot
fig, ax = pyplot.subplots(figsize=(6,6))
ax.plot(x_axis, results['validation_0']['mlogloss'], label='test')
ax.plot(x_axis, results['validation_0']['merror'], label='train')
ax.legend()

In [ ]:
from xgboost import plot_importance

fig, ax = pyplot.subplots()
plot_importance(loaded_model, ax=ax)
pyplot.show()

In [ ]:
from matplotlib import pyplot
# import pickle
# loaded_model = pickle.load(open("lpg_model.dat", "rb"))
results = clf.evals_result()
pyplot.plot(results['validation_0']['mlogloss'], label='test')
pyplot.plot(results['validation_1']['mlogloss'], label='train')
pyplot.ylabel("Log Loss")
# show the legend
pyplot.legend()
# show the plot
pyplot.show()

In [ ]:
pyplot.plot(results['validation_0']['merror'], label='test')
pyplot.plot(results['validation_1']['merror'], label='train')
pyplot.ylabel("Classification Error")
# show the legend
pyplot.legend()
# show the plot
pyplot.show()